In [1]:
### Code for the master thesis of Job Wegman

## this code will be divided in 3 stages

## first, the hyper parameters
## second, the perparation of the data
## third, the models and results

## hyperparameters

bigram_treshold = 70
keyword_number = 50
max_feats = 2000
k_neigbours = 7
bigram_enabled = True 


In [2]:
### Code for the master thesis of Job Wegman

## this code will be divided in 7 stages

from functions_job import dataset_shuffler
from functions_job import keywordextractor
from functions_job import make_bigram

import numpy as np
import pandas as pd
import pickle
import os
import json
import sys
#os.chdir('/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020')
print(os.getcwd())
ourwatched_set_filepath = '/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020/data'
## filepath of: which folder the pickled data should be stored
pickle_filepath = '/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020/pickles'
## filepath of: labels of transcripts from ~480 videos (Alfano, et al.)
dataset_filepath = '/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020/data'
## filepath of: transcripts from ~480 videos from (Alfano, et al.)
transcript_filepath = '/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020/data'

teamA_filepath = '/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020'
teamB_filepath = '/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020'


## shuffle datasets ##


## The training and test data both consist of the shape ([transcript, conspiracy rating]) ##

test_data, training_data = dataset_shuffler(ourwatched_set_filepath, pickle_filepath, dataset_filepath, transcript_filepath, teamA_filepath,  teamB_filepath)





from functions_job import cleaner
from functions_job import my_cleaner3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import en_core_web_sm

nlp = en_core_web_sm.load(disable=["parser", "tagger", "ner"])


training_data_cleaned = cleaner(training_data,nlp, pickle_filepath,'training_data_cleaned')
test_data_cleaned = cleaner(test_data,nlp, pickle_filepath,'test_data_cleaned')




## 2. in this part the bigrams are formed. ------------------------------------

if bigram_enabled == True:
    training_data_cleaned, test_data_cleaned = make_bigram(training_data_cleaned, test_data_cleaned, bigram_treshold)



    

## keywords extraction -------------------

filepath = pickle_filepath

keywordextractor(filepath,'training_data_cleaned.pickle',keyword_number)

import os
os.chdir(filepath)
import pickle
with open(str(keyword_number) + 'keyword_list1.pickle', 'rb') as f:
    keyword_list1 = pickle.load(f)
with open(str(keyword_number) + 'keyword_list2.pickle', 'rb') as f:
    keyword_list2 = pickle.load(f)







# Feature extraction


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


# for the train set I create X and y
X_trainingdata = []
y_trainingdata = []
for item in training_data_cleaned:
    X_trainingdata.append(item[0])
    y_trainingdata.append(item[1])

# Here I transform X to fit the input of the tfidf vectorizer
list_document_tokens = []
for i, document in enumerate(X_trainingdata):
    list_document_tokens.append(training_data_cleaned[i][0])
tfidf_input = []
for document in list_document_tokens:
    tfidf_input.append(" ".join(document))


# I do the same for my test data
X_testdata = []
y_testdata = []
for item in test_data_cleaned:
    X_testdata.append(item[0])
    y_testdata.append(item[1])

# once again making them fit
list_document_tokens_test = []
for i, document in enumerate(X_testdata):
    list_document_tokens_test.append(test_data_cleaned[i][0])
tfidf_input_test = []
for document in list_document_tokens_test:
    tfidf_input_test.append(" ".join(document))

    
### TF IDF VECTORIZATION


#for training I fit and transform the train data:
tv_training = TfidfVectorizer(stop_words=None, max_features=max_feats)
tf_idf_prel_training = tv_training.fit_transform(tfidf_input)
tf_idf_vector_training = tf_idf_prel_training.toarray()
tf_idf_feature_names_training = tv_training.get_feature_names()

#for test, using the same vectorizer I only transform the test data
tf_idf_prel_test = tv_training.transform(tfidf_input_test)
tf_idf_vector_test = tf_idf_prel_test.toarray()


### We route this back in X and Y variables (X_val & y_val for the validation stage, X_ts0 & y_tst for the final test set)
#
X_val = tf_idf_vector_training
y_val = []

# Here we condense class 2 & 3 into one class, leaving is with 0 (non conspiratorial) or 1 (conspiratorial)
for document in y_trainingdata:
    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val.append(class_made)

y_val = np.array(y_val)

X_tst = tf_idf_vector_test
y_tst = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst.append(class_made)

y_tst = np.array(y_tst)



### Here we set a baseline for the rest of the research


# train test split for validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_val, y_val, test_size=0.2, random_state=0)

X_bias1 = []

# we need gensims word2vec here
import gensim
import logging

# initialise and train the Word2Vec model
model = gensim.models.Word2Vec(list_document_tokens, size=150, window=10, min_count=0, workers=1, iter=10, seed = 0)


for w1 in tf_idf_feature_names_training:
    if w1 != 'pron' and w1 != 'to':
        counter = 0
        averager = 0
        for w2 in keyword_list1:
            
            try:
                counter += model.wv.similarity(w1, w2)
                #print(counter)
                averager += 1
            except:
                averager += 0
                pass

        #counter = counter/averager
        if counter > 0:
            counter += 1
        else:
            counter = counter
        X_bias1.append(counter)
    else:
        counter = 0
        X_bias1.append(counter)   
        

X_val_biased_1 = []
for i,matrixrow in enumerate(X_val):
    temp_num = []
    for j, item in enumerate(matrixrow):

        temp_num.append(X_bias1[j]*item)

    X_val_biased_1.append(temp_num)
import numpy as np
X_val_biased_1 = np.array(X_val_biased_1)

X_test_biased_1 = []
for i,matrixrow in enumerate(X_tst):
    temp_num = []
    for j, item in enumerate(matrixrow):

        temp_num.append(X_bias1[j]*item)

    X_test_biased_1.append(temp_num)
import numpy as np
X_test_biased_1 = np.array(X_test_biased_1)


# Now I route this back in X and Y variables
#for training:


X_val_bias1 = X_val_biased_1
y_val_bias1 = []
for document in y_trainingdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_val_bias1.append(class_made)

y_val_bias1 = np.array(y_val_bias1)

# for testing
X_tst_bias1 = X_test_biased_1
y_tst_bias1 = []
for document in y_testdata:

    class_made = 0
    if document == 1:
        class_made = 0
    else:
        class_made = 1
    y_tst_bias1.append(class_made)

y_tst_bias1 = np.array(y_tst_bias1)


# train test split for validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_val_bias1, y_val_bias1, test_size=0.2, random_state=0)

#print(len(X_train) + len(X_test))

/Users/ruud/Documents/DSS/thesis/JobWegman_Thesis2020
there are this many keywords in list1:  50
fully loaded
['government', 'conspiracy', 'vaccine', 'trump', 'freedom', 'abuse', 'speech', 'china', 'epstein', 'currency', 'federal', 'reptilians', 'woman', 'zuckerberg', 'cia', 'pedophile', 'wealth', 'pyramid', 'allege', 'darpa', 'president', 'lord', 'turkey', 'satanic', 'reptilian', 'council', 'gender', 'mainstream', 'bush', 'nation', 'intelligence', 'censor', 'consciousness', 'allegation', 'society', 'traveler', 'report', 'network', 'equality', 'trust', 'hierarchy', 'civilization', 'facebook', 'jordan', 'mariah', 'social', 'weinstein', 'agendum', 'bible', 'evidence']
finished extracting keywords


In [ ]:
## model 1: euclidean distance
#k_neigbours = 7
from sklearn.neighbors import KNeighborsClassifier
euc = KNeighborsClassifier(metric ='euclidean', p=2, n_neighbors= k_neigbours)
euc.fit(X_train, y_train)

# validation EUC

if bigram_enabled == True:
    print("euc val bigram")
else:
    print("euc val")


y_pred_euc = euc.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred_euc))
print(classification_report(y_test, y_pred_euc))
print(accuracy_score(y_test, y_pred_euc))



if bigram_enabled == True:
    print("euc test bigram")
else:
    print("euc test")
y_pred_euc_test = euc.predict(X_tst_bias1)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_tst_bias1, y_pred_euc_test))
print(classification_report(y_tst_bias1, y_pred_euc_test))
print(accuracy_score(y_tst_bias1, y_pred_euc_test))



## model 2: cosine distance

from matplotlib import pyplot as plt
from sklearn import datasets, metrics
from sklearn.neighbors import KNeighborsClassifier
from keras.datasets import mnist
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals import joblib
import pandas as pd
import time

cos = KNeighborsClassifier(metric ='cosine', p=2, n_neighbors= k_neigbours)
cos.fit(X_train, y_train)


# validation COS

if bigram_enabled == True:
    print("cos val bigram")
else:
    print("cos val")
y_pred_cos = cos.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred_cos))
print(classification_report(y_test, y_pred_cos))
print(accuracy_score(y_test, y_pred_cos))


if bigram_enabled == True:
    print("cos test bigram")
else:
    print("cos test")
y_pred_cos_test = cos.predict(X_tst_bias1)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_tst_bias1, y_pred_cos_test))
print(classification_report(y_tst_bias1, y_pred_cos_test))
print(accuracy_score(y_tst_bias1, y_pred_cos_test))



## model 3: DWC

from scipy.spatial import distance


def DWC_distance(string1, string2): 
    
    cossim = 1 - distance.cosine(string1, string2)
    distance1 = 0
    
    L = len(string1)
    for i in range(L):
        
        if np.sign([string1[i]]) != np.sign([string2[i]]):
            if string1[i] == 0 and string2[i] == 0:
                distance1 += 0
            elif string1[i] == 0:
                distance1 += string2[i]
            elif string2[i] == 0:
                distance1 += string1[i]
            else:
                distance1 += string1[i] * string2[i]
                
                
    
    return 1-(cossim / (cossim + (distance1/L) ))

DWC = KNeighborsClassifier(metric = DWC_distance, p=2, n_neighbors= k_neigbours)

DWC.fit(X_train, y_train)


# validation DWC

if bigram_enabled == True:
    print("dwc val bigram")
else:
    print("dwc val")

y_pred_DWC = DWC.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred_DWC))
print(classification_report(y_test, y_pred_DWC))
print(accuracy_score(y_test, y_pred_DWC))


if bigram_enabled == True:
    print("DWC test bigram")
else:
    print("DWC test")
y_pred_DWC_test = DWC.predict(X_tst_bias1)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_tst_bias1, y_pred_DWC_test))
print(classification_report(y_tst_bias1, y_pred_DWC_test))
print(accuracy_score(y_tst_bias1, y_pred_DWC_test))


euc val bigram
[[44  6]
 [ 9 17]]
              precision    recall  f1-score   support

           0       0.83      0.88      0.85        50
           1       0.74      0.65      0.69        26

    accuracy                           0.80        76
   macro avg       0.78      0.77      0.77        76
weighted avg       0.80      0.80      0.80        76

0.8026315789473685
0.77
euc test bigram
[[62  7]
 [11 16]]
              precision    recall  f1-score   support

           0       0.85      0.90      0.87        69
           1       0.70      0.59      0.64        27

    accuracy                           0.81        96
   macro avg       0.77      0.75      0.76        96
weighted avg       0.81      0.81      0.81        96

0.8125
0.76


Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


cos val bigram
[[44  6]
 [ 7 19]]
              precision    recall  f1-score   support

           0       0.86      0.88      0.87        50
           1       0.76      0.73      0.75        26

    accuracy                           0.83        76
   macro avg       0.81      0.81      0.81        76
weighted avg       0.83      0.83      0.83        76

0.8289473684210527
0.81
cos test bigram
[[57 12]
 [ 5 22]]
              precision    recall  f1-score   support

           0       0.92      0.83      0.87        69
           1       0.65      0.81      0.72        27

    accuracy                           0.82        96
   macro avg       0.78      0.82      0.80        96
weighted avg       0.84      0.82      0.83        96

0.8229166666666666
0.80
dwc val bigram
[[40 10]
 [ 5 21]]
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        50
           1       0.68      0.81      0.74        26

    accuracy                   